# TECHIN 510
## Milestone 1

**Name**: Saif Mustafa

**Email**: saifm@uw.edu

**Student Number**: 1428039

---

As part of the first milestone for your projects, you will practice what you have learned so far by developing the visual recognition capabilities of the robot you choose to develop. As a first step read the project topics listed below and decide which topic you would like to work on. You will choose and implement two visual recognition functionalities from the specs listed in the project topic you choose.

Your implementation will be in Python. You can use everything covered in class as well as any new functionality you discover yourself.

You will complete this assignment by submitting a demo video and any materials (code, data, physical prompts) that would allow us to recreate your demo, on Canvas by Oct 29, 2021 (Friday). The video should clearly illustrate two functionalities. Make sure you demonstrate the functionality in varied scenarios (e.g., for face detection make sure you have at least two different people's faces in different poses relative to the camera). At this point, your video does not need to include narrative about the project. It can be as simple as a screenshot video showing the captured images from a camera with annotations that reflect the implemented functionalities (e.g. square around faces for face detection).

---

## Topic: Nike Exercise and Wellness Robot

**Problem:** Many people who know the importance and potential benefits of exercising and meditation have a hard time motivating themselves to actually do them.

**Proposed solution:** Social robots have been demonstrated to have the impact of a social accountability partner in committing to difficult behavioral changes. This project will explore this potential of social robots for exercise, yoga, and/or meditation motivation and guidance.

**Prototype specifications:** The robot will have one user. The robot should interact with the user to introduce itself, meet its user, set user goals, and motivate the user to reach those goals. It should also guide the user through a sample exercise.

**Image processing capabilities for this robot (Milestone 1):**
- determine when a person is in front of the robot, 
- recognize whether the person is the owner of the robot, 
- determine the mood of the person, 
- determine when a person has completed an exercise, 
- determine when a functionality activation card (e.g. to start a specific exercise) is visually shown to the robot.

---

##  Installs / Imports

In [ ]:
# installing opencv and mediapipe https://google.github.io/mediapipe/
!pip install mediapipe
!pip install opencv-python
!pip install deepface
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace

mp_drawing = mp.solutions.drawing_utils # drawing model
mp_pose = mp.solutions.pose # pose estimation model

## Video Code + Basic detection

In [2]:
# find out my mac's webcam dimensions

vcap = cv2.VideoCapture(0) # 0=camera
 
if vcap.isOpened(): 
    # get vcap property 
    width  = vcap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`

    # it gives me 0.0 :/
    fps = vcap.get(cv2.CAP_PROP_FPS)
    
    print("width =",width)
    print("height =",height)
    
    # 1280 x 720

width = 1280.0
height = 720.0


In [2]:
# https://docs.opencv.org/master/dd/d43/tutorial_py_video_display.html

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()
    
# setting detection and tracking confidence    
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # Our operations on the frame come here
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        landmarks = results.pose_landmarks.landmark
        
        # basic detection using the draw_landmarks utility
        mp_drawing.draw_landmarks(image, # image
                                  results.pose_landmarks, # coordinates
                                  mp_pose.POSE_CONNECTIONS, # pose connections
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2), # dots
                                  mp_drawing.DrawingSpec(thickness=2, circle_radius=2)) # connections
        
        # Display the resulting frame
        cv2.imshow('NIKE WELLNESS DETECTOR', image)
        
        if cv2.waitKey(1) == ord('q'):
            break
    
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'landmark'

In [ ]:
# mp_drawing.draw_landmarks??

## Identifying joints

Mediapipe all usable body landmarks.

- 33 landmarks in total
- index starting at 0
- represent joints within the pose


![Mediapipe body landmarks](https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png)

In [ ]:
# Mappings
i = 0
for dot in mp_pose.PoseLandmark:
    print(i, "=", dot)
    i+=1

In [ ]:
# Getting the specific coordinates to a given landmark:
landmarks[mp_pose.PoseLandmark.NOSE.value] # nose
# or
landmarks[0]

In [ ]:
# measure curls -- need 11, 13, 15
landmarks[11] # left shoulder
landmarks[13] # left elbow
landmarks[15] # left wrist

In [ ]:
# mouth example
mouth = [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x,
         landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].y,
         landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].z]

mouth

In [ ]:
# angle needed between shoulder, elbow, and wrist to determine curl
# just doing x and y for now

shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]

wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

shoulder, elbow, wrist

get_angle(shoulder, elbow, wrist)

## Angle calculations

In [ ]:
# get angle between any 3 given points
def get_angle(p1, p2, p3):
    
    p1 = np.array(p1)
    p2 = np.array(p2)
    p3 = np.array(p3)
    
    radians = np.arctan2(p3[1]-p2[1], p3[0]-p2[0]) - np.arctan2(p1[1]-p2[1], p1[0]-p2[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

## Building a counter / tracker

In [ ]:
# https://docs.opencv.org/master/dd/d43/tutorial_py_video_display.html

cap = cv2.VideoCapture(0)

print("Saif")

if not cap.isOpened():
    print("Cannot open camera")
    exit()
    
curl_count=0
curl_flag_left="down"
curl_flag_right="False"
    
# setting detection and tracking confidence    
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        # Our operations on the frame come here
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract all joints
        try:
            landmarks = results.pose_landmarks.landmark
            
            # shoulder coordinates
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            #print("Shoulder left =",shoulder_left)
            
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            #print("Shoulder right =",shoulder_right)
            
            # elbow coordinates
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            # wrist coordinates
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # calculate angle
            # logic:
            # - if angle is > 90, then it's a curl down
            # - if angle < 90, the it's a curl up
            # - some kind of counter that starts and countfs every 2 as 1 successful curl
            # - visualize on screen
            curl_angle_left = get_angle(shoulder_left, elbow_left, wrist_left)
            #print("Curl angle left =", curl_angle_left)
            curl_angle_right = get_angle(shoulder_right, elbow_right, wrist_right)
            #print("Curl angle right =", curl_angle_right)
            
            # show angle at elbow
            cv2.putText(image, 
                        str(round(curl_angle_right)), 
                        tuple(np.multiply(elbow_right, [1280,720]).astype(int)),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)

            cv2.putText(image, 
                        str(round(curl_angle_left)), 
                        tuple(np.multiply(elbow_left, [1280,720]).astype(int)),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
            
            # print("Count = ",curl_count)
            
            if curl_angle_left > 160:
                curl_flag_left = False # curl down
            if curl_angle_left < 50 and not curl_flag_left:
                curl_flag_left = True # curl up
                curl_count+=1
                print("Count inside = ", curl_count)
                
            if curl_angle_right > 160:
                curl_flag_right=False # curl down
            if curl_angle_right < 50 and not curl_flag_right:
                curl_flag_right=True # curl up
                curl_count+=1
                print("Count inside = ", curl_count)
            
        except:
            pass
        
        # display count
        #cv2.rectangle(image, start_point, end_point, color, thickness)
        #cv2.rectangle(image,(0,620),(1280,720),(0,0,0),-1)
        cv2.putText(image, "NIKE FITNESS TRACKER", (500,600),
                    cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        cv2.rectangle(image, (600,610), (700,710), (69,255,213), -1) # rgba(213,255,69,255)
        cv2.putText(image, "REPS", (635,635),
                    cv2.FONT_HERSHEY_DUPLEX, .4, (120,116,124), 1, cv2.LINE_AA) # rgb(124,116,120)
        cv2.putText(image, str(curl_count), (640,675),
                    cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA) # rgb(124,116,120)
        
        
        # basic detection using rhe draw_landmarks utility
        mp_drawing.draw_landmarks(image, # image
                                  results.pose_landmarks, # coordinates
                                  mp_pose.POSE_CONNECTIONS, # pose connections
                                  mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2), # dots
                                  mp_drawing.DrawingSpec(thickness=2, circle_radius=2)) # connections
        
        
        # Display the resulting frame
        cv2.imshow('NIKE WELLNESS DETECTOR', image)
        
        if cv2.waitKey(1) == ord('q'):
            break
    
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

Saif


In [ ]:
# 